# Some other race

Predictions:

1. The upcoming propsal for changing the census is essentially adding hispanic as a race (because the ethnicity question is being eliminated)
2. Middle Eastern/North African

This means that the categories (minimum categories) will be:
1. American Indian/Alaska Native
2. Asian
3. Black/African American
4. Hispanic/Latino
5. Middle Eastern/North African
6. Native Hawaiian / Pacific Islander
7. White

What would be good for predictions?
1. Some maps--looking at the ACS data to see where people fall under Hispanic and thus predict a shift in responses for 2030 ethnicity
2. Looking specifically at country level to determine how adding Middle Eastern/North African will impact the responses as well. 

## Importing

In [11]:
import json

import geopandas as gpd
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import requests
import us

from census import Census
from shapely.geometry import Point

import plotly.express as px

from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

In [4]:
census = Census("", year=2020)

# Read files

In [99]:
percents = pd.read_csv('race_pct_by_county.csv')

In [101]:
percents = percents.astype({'GEOID': 'str'})

In [103]:
percents.iloc[0]['GEOID'] 

'1001'

In [104]:
string_fips = ['0']*len(percents)
for i in range(len(percents)):
    if len(percents.iloc[i]['GEOID']) == 4: 
        string_fips[i] = '0' + percents.iloc[i]['GEOID']
    else: 
        string_fips[i] = percents.iloc[i]['GEOID']
percents['GEOID'] = string_fips

In [107]:
percents['GEOID']

0       01001
1       01003
2       01005
3       01007
4       01009
        ...  
3216    27129
3217    27135
3218    27141
3219    27147
3220    27153
Name: GEOID, Length: 3221, dtype: object

## Functions

In [113]:
def fips_to_string(dataframe, fips_col_name):
    dataframe = dataframe.astype({fips_col_name: 'str'})
    string_fips = ['0']*len(dataframe)
    for i in range(len(dataframe)):
        if len(dataframe.iloc[i][fips_col_name]) == 4: 
            dataframe[i] = '0' + dataframe.iloc[i][fips_col_name]
        else: 
            string_fips[i] = dataframe.iloc[i][fips_col_name]
    dataframe[fips_col_name] = string_fips
    return dataframe

In [123]:
def make_heatmap(dataframe, fips_column_name, column_name):
    maximum = dataframe[column_name].max()
    fig = px.choropleth(dataframe, geojson=counties, locations=fips_column_name, color=column_name,
                           color_continuous_scale="Viridis",
                           range_color=(0, maximum),
                           scope="usa",
                           labels={'white':'percent white pop'}
                          )
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    fig.show()

## Visualizations

In [37]:
counts = pd.read_csv('dataframe_stats_and_shape_w_ct_counties.csv')

In [39]:
type(counts.iloc[0]['GEOID'])

numpy.int64

In [121]:
percents['white'].max()

3259427.0

In [115]:
categories = [
    "white_pct",
    "black_pct",
    "amin_pct",
    "asian_pct",
    "nhpi_pct",
    "other_pct",
    "two_or_more_pct",
    "nh_white_pct",
    "nh_black_pct",
    "nh_amin_pct",
    "nh_asian_pct",
    "nh_nhpi_pct",
    "nh_other_pct",
    "nh_two_or_more_pct"
]

In [ ]:
for category in categories:
    make_heatmap(percents, 'GEOID', category)